# Transformers - How things came to here!

Several years back, before deep learning taking a massive outbreak most of the natural language processing tasks were stuck at a very primitive level. After the DL initial usages, RNNs and CNNs came into the scene and improved the models we used to do the NLP tasks greatly. But none of them affected the NLP world as the Transformers architecture which was introduced by a google researcher team.

In early days of NLP, Markov decision processes, markov models, chains were used to predict the next element of a sequence given the past elements. Then these turned into the space of CNNs. But CNNs have the drawback of not keeping track of sequences in long/complex inputs. Then came the RNNs and LSTMs to mitigate the above issues which can keep a state within. 

After all those, in 2017 google research/brain scientists published a paper `Attention is all you need` which introduces the architecture of Transformers. 

For the moment I will not go in to the details of the model architecture of the original transformer. But important point to know is attention mechanism replaced the recurrence which need to have high computational power to keep details of long sequences. Instead attention is a `word to word` operation. The attention mechanism will find how each word is related to each other including itself. Therefore it can provide deeper relationships between words and produce better results.

<center><image src="imgs/1.png" width="400"/></center>

Normal Transformer architecture consists with 2 major parts encoder and decoder. Usually these part consists of stacks of layers.

### **Encoder stack**

The original transformer's encoder stack consists of 6 layers of below stucture.

<center><image src="imgs/2.jpg" width="200"/></center>

Each of above like structure contains 2 sub layers: a multiheaded attention mechanism and a fully connected(dense) position-wise feedforward network. Also note the residual connections (skip connections) in each such sublayers. This make sure all the key information such as positional encoding is not lost on the way. (Note Embedding is only available to the first one.)

#### **Input Embedding**

In the original paper this is of size 512. As with any other model embedding layer, usage of this is pretty simple. Assume we have a sentence like below.

<center>My name is Dilan and I am a software engineer.</center>

Then we need to tokenize it.

<center>[My, name, is, Dilan, and, I, am, a, software, engineer, .]</center>

Since we cant use text in processes, we will assign integer values for each token.

<center>[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]</center>

Then using these integers we can define a vocabulary and move in to embedding training. To do that we define a (vocabulary x defined width) sized matrix as embedding matrix. Then we use it in the first section of NLP model. To see a example implementation check this [Jupyter Notebook of Mine](https://github.com/DinushkaDDS/DL-models-from-Scratch/blob/master/sentence%20embedding.ipynb).


There are many architecures to train a embedding layer, depending on the task like `word2vec` skipgram and ngram. 



